# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 8.3.0
-- The CXX compiler identification is GNU 8.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/tdelgado/anaconda3/bin/python (found version "3.7.3") 
-- Found PythonLibs: /home/tdelgado/anaconda3/lib/libpython3.7m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/tdelgado/Desktop/Tp2Metodos
-- Configuring done
-- Generatin

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/tdelgado/Desktop/Tp2Metodos/notebooks
Python 3.7.3


In [12]:
USAR_IMDB_LARGE = False
SIZE_TRAIN = 5000
SIZE_TEST = 500
DATA_SIZE = 3000 #CANTIDAD DE REVIEWS A LEER DEL DATASET, LO PONGO PORQUE 50k PUEDE TARDAR
                 #LA CANTIDAD TOTAL ES DATA_SIZE*4 PORQUE TOMA TEST TRAIN POS Y NEG

In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os

!cd ../data && tar -xvf *.tar.gz

if not USAR_IMDB_LARGE:
    df = pd.read_csv("../data/imdb_small.csv", index_col=0) 
    df = df[['review','label']]
    df['label'] = (df['label'] == 'pos').astype('int')
else:
    folder = '../data/aclImdb'
    labels = {'pos': 1, 'neg': 0}
    df = pd.DataFrame()
    for f in ('test', 'train'):    
        for l in ('pos', 'neg'):
            path = os.path.join(folder, f, l)
            c = 0
            for file in os.listdir (path) :
                with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                    txt = infile.read()
                df = df.append([[txt, labels[l]]],ignore_index=True)
                c+=1
                if c == DATA_SIZE:
                    break
    df.columns = ['review', 'label']
df.describe()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
imdb_small.csv


,label
count,12500.000000
mean,0.494240
std,0.499987
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [14]:
import numpy as np

X = np.array(df['review'].tolist())
y = np.array(df['label'].tolist())
np.random.shuffle(X)
np.random.shuffle(y)

X_train = X[:SIZE_TRAIN]
y_train = y[:SIZE_TRAIN]
X_test = X[SIZE_TRAIN:SIZE_TRAIN+SIZE_TEST]
y_test = y[SIZE_TRAIN:SIZE_TRAIN+SIZE_TEST]

print("Cantidad de instancias de entrenamiento = {}".format(len(X_train)))
print("Cantidad de instancias de test = {}".format(len(X_test)))

print("Class balance : {} pos {} neg".format(
    y_train.sum() / y_train.shape[0], 
    (y_train.shape[0] - y_train.sum()) / y_train.shape[0]
))

Cantidad de instancias de entrenamiento = 5000
Cantidad de instancias de test = 500
Class balance : 0.4902 pos 0.5098 neg


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

vectorizer = TfidfVectorizer(ngram_range = (1,3), use_idf = True, smooth_idf = True, max_df = 0.9, min_df = 0.01)
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [16]:
ALFA = 500
print(X_train.shape)
pca_transformer = sentiment.PCA(ALFA)
pca_transformer.fit(X_train.todense())
X_train = pca_transformer.transform(X_train)
X_test = pca_transformer.transform(X_test)

(5000, 3884)


In [17]:
K = 20

clf = sentiment.KNNClassifier(K)

clf.fit(X_train, y_train)


In [18]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.506


In [ ]:
print(vectorizer.vocabulary_)